<a href="https://colab.research.google.com/github/seecode4/seeRepo1/blob/main/capstone/research_repro/bidaf_triviaqa_squad_try1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Capstone Step4 - Try [triviaqa](https://github.com/mandarjoshi90/triviaqa) - BiDAF Model  

The BiDAF model used in the paper, [TriviaQA: A Large Scale Distantly Supervised Challenge Dataset for Reading Comprehension](https://arxiv.org/abs/1705.03551), is based on the original paper for [Bidirectional Attention Flow for Machine Comprehension](https://arxiv.org/abs/1611.01603).  

Downloaded SQuAD data and GloVe and nltk corpus. Then followed steps outlined in the related git reference https://github.com/allenai/bi-att-flow/  

Got the following and was unable to proceed further.  
  W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT  
It appears that TensorRT is present but it is unable to access it.


In [1]:
# Ref: https://stackoverflow.com/questions/63827064/git-folder-to-google-colab
# Decentralized git clone from https://github.com/allenai/bi-att-flow/
# Example in https://github.com/fredzett/rmqa/blob/master/S02c_Exercise1.ipynb
#   !npx degit fredzett/rmqa/data data
!pwd
!ls

/content
drive  sample_data


## 0. Requirements
#### General
- Python (verified on 3.5.2. Issues have been reported with Python 2!)
- unzip, wget (for running `download.sh` only)

#### Python Packages
- tensorflow (deep learning library, only works on r0.11)
- nltk (NLP tools, verified on 3.2.1)
- tqdm (progress bar, verified on 4.7.4)
- jinja2 (for visaulization; if you only train and test, not needed)


In [2]:
"""
https://github.com/allenai/bi-att-flow/issues/81
https://github.com/tensorflow/tensorflow/issues/64809
"""
!pip install tensorflow
!pip install TensorRT
import tensorflow as tf

In [3]:
# Ref: https://github.com/allenai/bi-att-flow/
# NPX is a command-line tool meant to run NPM packages containing CLI scripts.
#  if it doesn't work you can always run npm install -g npx
#  degit user/repo/subdirectory

# Done once and saved to GDrive
# !npx degit allenai/bi-att-flow trivia_qa_bidaf_orig

In [4]:
# Click folder icon on left ; mount drive and copy trivia_qa_bidaf_orig to /content/drive/Colab Notebooks
!ls
# !ls -lrt "/content/drive/MyDrive/Colab Notebooks"
!ls "/content/drive/MyDrive/GDcc4/ColabNotebooks"
%cd "/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone"
!ls -lrt

drive  sample_data
cc4_capstone  cc4_curriculum
/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone
total 781
drwx------  2 root root   4096 Aug 11 22:14 trivia_qa_orig
drwx------  2 root root   4096 Aug 12 17:29 see_trivia_qa_try1
-rw-------  1 root root    174 Aug 13 18:24 see_trivia_qa_notes.gdoc
-rw-------  1 root root   9073 Aug 15 22:33 trivia_qa_read_data.ipynb
-rw-------  1 root root  14326 Aug 16 04:14 see_trivia_qa_sample_colors.ipynb
-rw-------  1 root root 698468 Aug 17 04:34 see_trivia_qa_read_drive.ipynb
drwx------ 10 root root   4096 Aug 18 21:39 trivia_qa_bidaf_orig
drwx------  2 root root   4096 Aug 31 01:05 SQuADv1_1
-rw-------  1 root root  55823 Oct  8 01:46 trivia_qa_get_entries_ijson.ipynb
drwx------  2 root root   4096 Oct  8 22:28 see_notes_save


In [5]:
# https://github.com/allenai/bi-att-flow/blob/master/README.md
!cat trivia_qa_bidaf_orig/README.md

# Bi-directional Attention Flow for Machine Comprehension
 
- This the original implementation of [Bi-directional Attention Flow for Machine Comprehension][paper].
- The CodaLab worksheet for the [SQuAD Leaderboard][squad] submission is available [here][worksheet].
- For TensorFlow v1.2 compatible version, see the [dev][dev] branch.
- Please contact [Minjoon Seo][minjoon] ([@seominjoon][minjoon-github]) for questions and suggestions.

## 0. Requirements
#### General
- Python (verified on 3.5.2. Issues have been reported with Python 2!)
- unzip, wget (for running `download.sh` only)

#### Python Packages
- tensorflow (deep learning library, only works on r0.11)
- nltk (NLP tools, verified on 3.2.1)
- tqdm (progress bar, verified on 4.7.4)
- jinja2 (for visaulization; if you only train and test, not needed)

## 1. Pre-processing
First, prepare data. Donwload SQuAD data and GloVe and nltk corpus
(~850 MB, this will download files to `$HOME/data`):
```
chmod +x download.sh; ./download.sh
`

In [6]:
# Go to directory to try
!pwd
!ls -lrt trivia_qa_bidaf_orig
%cd trivia_qa_bidaf_orig

/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone
total 60
-rw------- 1 root root    33 Jan 25  2018  requirements.txt
-rw------- 1 root root  6576 Jan 25  2018  README.md
-rw------- 1 root root 11358 Jan 25  2018  LICENSE
-rw------- 1 root root   695 Jan 25  2018  download.sh
drwx------ 2 root root  4096 Aug 18 21:39  visualization
drwx------ 2 root root  4096 Aug 18 21:39  tree
drwx------ 3 root root  4096 Aug 18 21:39  squad
drwx------ 5 root root  4096 Aug 18 21:39  my
drwx------ 2 root root  4096 Aug 18 21:39  cnn_dm
drwx------ 2 root root  4096 Aug 18 21:39  basic_cnn
drwx------ 4 root root  4096 Aug 18 21:39  basic
-rw------- 1 root root  7757 Aug 18 22:54 'Copy of nn.py'
drwx------ 5 root root  4096 Oct  9 00:31  data
/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig


In [7]:
!pwd
!ls -lrt
print("!ls -lrt data")
!ls -lrt data
print("!ls -lrt data/squad")
!ls -lrt data/squad
print("!du -hs data/squad")
!du -hs data/squad

/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig
total 60
-rw------- 1 root root    33 Jan 25  2018  requirements.txt
-rw------- 1 root root  6576 Jan 25  2018  README.md
-rw------- 1 root root 11358 Jan 25  2018  LICENSE
-rw------- 1 root root   695 Jan 25  2018  download.sh
drwx------ 2 root root  4096 Aug 18 21:39  visualization
drwx------ 2 root root  4096 Aug 18 21:39  tree
drwx------ 3 root root  4096 Aug 18 21:39  squad
drwx------ 5 root root  4096 Aug 18 21:39  my
drwx------ 2 root root  4096 Aug 18 21:39  cnn_dm
drwx------ 2 root root  4096 Aug 18 21:39  basic_cnn
drwx------ 4 root root  4096 Aug 18 21:39  basic
-rw------- 1 root root  7757 Aug 18 22:54 'Copy of nn.py'
drwx------ 5 root root  4096 Oct  9 00:31  data
!ls -lrt data
total 13
-rw------- 1 root root  207 Oct  9 00:31 readme.txt
drwx------ 2 root root 4096 Oct  9 00:31 glove
drwx------ 2 root root 4096 Oct  9 00:31 squad_1_1
drwx------ 2 root root 4096 Oct  9 01:16 squad
!ls -lrt data/sq

In [8]:
!echo $HOME
!echo $PWD
print("ls /root")
!ls /root
!pwd
!ls ../SQuADv1_1

/root
/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig
ls /root
data  nltk_data
/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig
readme_sk.txt.txt


In [9]:
%%time
## 1. Pre-processing - skip data is in GDrive cc4_capstone/data_local
# First, prepare data. Donwload SQuAD data and GloVe and nltk corpus
# (~850 MB, this will download files to `$HOME/data`):
# !chmod +x download.sh
# !./download.sh
!cat ./download.sh

#!/usr/bin/env bash

DATA_DIR=$HOME/data
mkdir $DATA_DIR

# Download SQuAD
SQUAD_DIR=$DATA_DIR/squad
mkdir $SQUAD_DIR
wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json -O $SQUAD_DIR/train-v1.1.json
wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json -O $SQUAD_DIR/dev-v1.1.json


# Download CNN and DailyMail
# Download at: http://cs.nyu.edu/~kcho/DMQA/


# Download GloVe
GLOVE_DIR=$DATA_DIR/glove
mkdir $GLOVE_DIR
wget http://nlp.stanford.edu/data/glove.6B.zip -O $GLOVE_DIR/glove.6B.zip
unzip $GLOVE_DIR/glove.6B.zip -d $GLOVE_DIR

# Download NLTK (for tokenizer)
# Make sure that nltk is installed!
python3 -m nltk.downloader -d $HOME/nltk_data punkt
CPU times: user 4.67 ms, sys: 1.09 ms, total: 5.77 ms
Wall time: 103 ms


In [10]:
# Saved to local GDrive # Download GloVe
!pwd
!ls data/glove
!echo wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip -O data/glove/glove.6B.zip > data/glove/readme.txt
# !wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip -O data/glove/glove.6B.zip

/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig
glove.6B.zip


In [11]:
# Saved to local GDrive # Download SQuAD
!echo wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json -O data/squad_1_1/train-v1.1.json >> data/glove/readme.txt
!echo wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json -O data/squad_1_/dev-v1.1.json >> data/glove/readme.txt
!mv data/glove/readme.txt data/readme.txt
!cat data/readme.txt
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json -O data/squad_1_1/train-v1.1.json
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json -O data/squad_1_1/dev-v1.1.json

wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip -O data/glove/glove.6B.zip
wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json -O data/squad_1_1/train-v1.1.json
wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json -O data/squad_1_/dev-v1.1.json


In [12]:
!ls -lrt data/glove data/squad_1_1
!echo $HOME
!echo $HOME/data
# !mkdir -p /root/data/squad /root/data/glove
# !ls /root/data/{squad,glove}
# !cp -p data/squad_1_1/* /root/data/squad
# !cp -p data/glove/* /root/data/glove
# !unzip /root/data/glove/glove.6B.zip -d /root/data/glove
!ls -lrt /root/data/{squad,glove}

data/squad_1_1:
total 34320
-rw------- 1 root root 30288272 Oct 13  2023 train-v1.1.json
-rw------- 1 root root  4854279 Oct 13  2023 dev-v1.1.json

data/glove:
total 841976
-rw------- 1 root root 862182613 Oct 25  2015 glove.6B.zip
/root
/root/data
/root/data/squad:
total 34324
-rw------- 1 root root 30288272 Oct 13  2023 train-v1.1.json
-rw------- 1 root root  4854279 Oct 13  2023 dev-v1.1.json

/root/data/glove:
total 3039132
-rw-rw-r-- 1 root root  347116733 Aug  4  2014 glove.6B.100d.txt
-rw-rw-r-- 1 root root  693432828 Aug  4  2014 glove.6B.200d.txt
-rw-rw-r-- 1 root root  171350079 Aug  4  2014 glove.6B.50d.txt
-rw-rw-r-- 1 root root 1037962819 Aug 27  2014 glove.6B.300d.txt
-rw------- 1 root root  862182613 Oct 25  2015 glove.6B.zip


In [13]:
# Download NLTK (for tokenizer)
import nltk
# dir(nltk)
# !python3 -m nltk.downloader -d $HOME/nltk_data punkt - skip this
nltk.download('punkt', download_dir='/root/nltk_data')
!ls -lrt /root /root/data /root/nltk_data/*

/root/data:
total 8
drwxr-xr-x 2 root root 4096 Oct  9 01:00 squad
drwxr-xr-x 2 root root 4096 Oct  9 01:03 glove

/root/nltk_data/tokenizers:
total 13584
-rw-r--r-- 1 root root 13905355 Oct  9 01:07 punkt.zip
drwxr-xr-x 3 root root     4096 Oct  9 01:08 punkt

/root:
total 8
drwxr-xr-x 4 root root 4096 Oct  9 00:42 data
drwxr-xr-x 3 root root 4096 Oct  9 01:07 nltk_data


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Notes from first try of download.sh - copies data to /root
--2024-08-19 23:09:42--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘/root/data/squad/train-v1.1.json’

/root/data/squad/tr 100%[===================>]  28.88M   171MB/s    in 0.2s    

2024-08-19 23:09:43 (171 MB/s) - ‘/root/data/squad/train-v1.1.json’ saved [30288272/30288272]

--2024-08-19 23:09:43--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘/root/data/squad/dev-v1.1.json’

/root/data/squad/de 100%[===================>]   4.63M  --.-KB/s    in 0.09s   

2024-08-19 23:09:43 (53.1 MB/s) - ‘/root/data/squad/dev-v1.1.json’ saved [4854279/4854279]

--2024-08-19 23:09:43--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-08-19 23:09:43--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-08-19 23:09:44--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘/root/data/glove/glove.6B.zip’

/root/data/glove/gl 100%[===================>] 822.24M  5.02MB/s    in 2m 39s  

2024-08-19 23:12:23 (5.18 MB/s) - ‘/root/data/glove/glove.6B.zip’ saved [862182613/862182613]

Archive:  /root/data/glove/glove.6B.zip
  inflating: /root/data/glove/glove.6B.50d.txt  
  inflating: /root/data/glove/glove.6B.100d.txt  
  inflating: /root/data/glove/glove.6B.200d.txt  
  inflating: /root/data/glove/glove.6B.300d.txt  
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

In [14]:
print("!ls /root")
!ls /root
print("\n!ls /root/data")
!ls /root/data
print("\n!ls /root/nltk_data")
!ls /root/nltk_data

!ls /root
data  nltk_data

!ls /root/data
glove  squad

!ls /root/nltk_data
tokenizers


In [15]:
!pwd
#!mv ./data /root
# Note squad.prepro expect train-v1.1.json etc in /root/data
!echo $PWD
!ls -l /root/data/squad
!du -hs /root/data

/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig
/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig
total 34324
-rw------- 1 root root  4854279 Oct 13  2023 dev-v1.1.json
-rw------- 1 root root 30288272 Oct 13  2023 train-v1.1.json
3.0G	/root/data


In [16]:
%%time
# Second, Preprocess Stanford QA dataset (along with GloVe vectors) and
# save them in `$PWD/data/squad` (~5 minutes):
!pwd
!ls -lrt /root/data
!ls -lrt /root/data/glove
!ls -lrt /root/data/squad

/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig
total 8
drwxr-xr-x 2 root root 4096 Oct  9 01:00 squad
drwxr-xr-x 2 root root 4096 Oct  9 01:03 glove
total 3039132
-rw-rw-r-- 1 root root  347116733 Aug  4  2014 glove.6B.100d.txt
-rw-rw-r-- 1 root root  693432828 Aug  4  2014 glove.6B.200d.txt
-rw-rw-r-- 1 root root  171350079 Aug  4  2014 glove.6B.50d.txt
-rw-rw-r-- 1 root root 1037962819 Aug 27  2014 glove.6B.300d.txt
-rw------- 1 root root  862182613 Oct 25  2015 glove.6B.zip
total 34324
-rw------- 1 root root 30288272 Oct 13  2023 train-v1.1.json
-rw------- 1 root root  4854279 Oct 13  2023 dev-v1.1.json
CPU times: user 20.1 ms, sys: 3.16 ms, total: 23.3 ms
Wall time: 432 ms


In [17]:
%%time
# Second, Preprocess Stanford QA dataset (along with GloVe vectors) and
# save them in `$PWD/data/squad` (~5 minutes):
!pwd
!python -m squad.prepro

/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig
100% 442/442 [01:17<00:00,  5.73it/s]
100% 400000/400000 [00:07<00:00, 53448.11it/s]
69844/103462 of word vocab have corresponding vectors in /root/data/glove/glove.6B.100d.txt
100% 400000/400000 [00:09<00:00, 43275.01it/s]
70717/90346 of word vocab have corresponding vectors in /root/data/glove/glove.6B.100d.txt
saving ...
100% 48/48 [00:17<00:00,  2.71it/s]
100% 400000/400000 [00:08<00:00, 48638.20it/s]
22749/28017 of word vocab have corresponding vectors in /root/data/glove/glove.6B.100d.txt
100% 400000/400000 [00:10<00:00, 36427.81it/s]
22905/24914 of word vocab have corresponding vectors in /root/data/glove/glove.6B.100d.txt
saving ...
100% 48/48 [00:15<00:00,  3.05it/s]
100% 400000/400000 [00:09<00:00, 43929.86it/s]
22749/28017 of word vocab have corresponding vectors in /root/data/glove/glove.6B.100d.txt
100% 400000/400000 [00:06<00:00, 57548.88it/s]
22905/24914 of word vocab have corresponding vectors

In [18]:
!ls -lrt /root/data
!ls -lrt /root/data/glove
!ls -lrt /root/data/squad

total 8
drwxr-xr-x 2 root root 4096 Oct  9 01:00 squad
drwxr-xr-x 2 root root 4096 Oct  9 01:03 glove
total 3039132
-rw-rw-r-- 1 root root  347116733 Aug  4  2014 glove.6B.100d.txt
-rw-rw-r-- 1 root root  693432828 Aug  4  2014 glove.6B.200d.txt
-rw-rw-r-- 1 root root  171350079 Aug  4  2014 glove.6B.50d.txt
-rw-rw-r-- 1 root root 1037962819 Aug 27  2014 glove.6B.300d.txt
-rw------- 1 root root  862182613 Oct 25  2015 glove.6B.zip
total 34324
-rw------- 1 root root 30288272 Oct 13  2023 train-v1.1.json
-rw------- 1 root root  4854279 Oct 13  2023 dev-v1.1.json


## 2. Training
The model has ~2.5M parameters.
The model was trained with NVidia Titan X (Pascal Architecture, 2016).
The model requires at least 12GB of GPU RAM.
If your GPU RAM is smaller than 12GB, you can either decrease batch size (performance might degrade),
or you can use multi GPU (see below).
The training converges at ~18k steps, and it took ~4s per step (i.e. ~20 hours).

Before training, it is recommended to first try the following code to verify everything is okay and memory is sufficient:
```
python -m basic.cli --mode train --noload --debug
```

In [19]:
# Ref: https://github.com/tensorflow/tensorflow/issues/64809
# https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html
# !python -c "import tensorrt; print(tensorrt.__file__)"
# !python -c "import tensorflow as tf; print(tf.config.list_physical_devices())"
!python -c "import tensorrt; print(dir(tensorrt), '\n', tensorrt.__version__)"
help_dir_tensorrt = ['APILanguage', 'ActivationType', 'AllocatorFlag',
                      'BoundingBoxFormat', 'Builder', 'BuilderFlag',
                      'CalibrationAlgoType', 'DataType', 'DeviceType',
                      'DimensionOperation', 'Dims', 'Dims2', 'Dims3', 'Dims4',
                      'DimsExprs', 'DimsHW', 'DynamicPluginTensorDesc',
                      'ElementWiseOperation', 'EngineCapability', 'EngineInspector',
                      'ErrorCode', 'ErrorCodeTRT', 'ExecutionContextAllocationStrategy',
                      'FallbackString', 'FillOperation', 'GatherMode', 'HardwareCompatibilityLevel',
                      'IActivationLayer', 'IAlgorithm', 'IAlgorithmContext', 'IAlgorithmIOInfo',
                      'IAlgorithmSelector', 'IAlgorithmVariant', 'IAssertionLayer', 'IBuilderConfig',
                      'ICastLayer', 'IConcatenationLayer', 'IConditionLayer', 'IConstantLayer', 'IConvolutionLayer',
                      'ICudaEngine', 'IDebugListener', 'IDeconvolutionLayer', 'IDequantizeLayer', 'IDimensionExpr',
                      'IEinsumLayer', 'IElementWiseLayer', 'IErrorRecorder', 'IExecutionContext', 'IExprBuilder',
                      'IFillLayer', 'IGatherLayer', 'IGpuAllocator', 'IGpuAsyncAllocator', 'IGridSampleLayer',
                      'IHostMemory', 'IIdentityLayer', 'IIfConditional', 'IIfConditionalBoundaryLayer',
                      'IIfConditionalInputLayer', 'IIfConditionalOutputLayer', 'IInt8Calibrator',
                      'IInt8EntropyCalibrator', 'IInt8EntropyCalibrator2', 'IInt8LegacyCalibrator',
                      'IInt8MinMaxCalibrator', 'IIteratorLayer', 'ILRNLayer', 'ILayer', 'ILogger',
                      'ILoop', 'ILoopBoundaryLayer', 'ILoopOutputLayer', 'IMatrixMultiplyLayer', 'INMSLayer',
                      'INetworkDefinition', 'INonZeroLayer', 'INormalizationLayer', 'IOneHotLayer', 'IOptimizationProfile',
                      'IOutputAllocator', 'IPaddingLayer', 'IParametricReLULayer', 'IPluginCapability', 'IPluginCreator',
                      'IPluginCreatorInterface', 'IPluginCreatorV3One', 'IPluginRegistry', 'IPluginResource',
                      'IPluginResourceContext', 'IPluginV2', 'IPluginV2DynamicExt', 'IPluginV2DynamicExtBase',
                      'IPluginV2Ext', 'IPluginV2Layer', 'IPluginV3', 'IPluginV3Layer', 'IPluginV3OneBuild',
                      'IPluginV3OneBuildV2', 'IPluginV3OneCore', 'IPluginV3OneRuntime', 'IPoolingLayer', 'IProfiler',
                      'IProgressMonitor', 'IQuantizeLayer', 'IRaggedSoftMaxLayer', 'IRecurrenceLayer', 'IReduceLayer',
                      'IResizeLayer', 'IReverseSequenceLayer', 'IScaleLayer', 'IScatterLayer', 'ISelectLayer',
                      'ISerializationConfig', 'IShapeLayer', 'IShuffleLayer', 'ISliceLayer', 'ISoftMaxLayer',
                      'IStreamReader', 'ITensor', 'ITimingCache', 'ITopKLayer', 'ITripLimitLayer', 'IUnaryLayer',
                      'IVersionedInterface', 'InterfaceInfo', 'InterpolationMode', 'LayerInformationFormat',
                      'LayerType', 'Logger', 'LoopOutput', 'MatrixOperation', 'MemoryPoolType',
                      'NetworkDefinitionCreationFlag', 'NodeIndices', 'OnnxParser', 'OnnxParserFlag',
                      'OnnxParserRefitter', 'PaddingMode', 'ParserError', 'Permutation',
                      'PluginCapabilityType', 'PluginCreatorVersion', 'PluginField', 'PluginFieldCollection',
                      'PluginFieldCollection_', 'PluginFieldType', 'PluginTensorDesc', 'PoolingType', 'PreviewFeature',
                      'Profiler', 'ProfilingVerbosity', 'QuantizationFlag', 'ReduceOperation', 'Refitter',
                      'ResizeCoordinateTransformation', 'ResizeRoundMode', 'ResizeSelector', 'Runtime',
                      'RuntimePlatform', 'SampleMode', 'ScaleMode', 'ScatterMode', 'SerializationFlag',
                      'SubGraphCollection', 'TacticSource', 'TempfileControlFlag',
                      'TensorFormat', 'TensorIOMode', 'TensorLocation', 'TensorRTPhase',
                      'TopKOperation', 'TripLimit', 'UnaryOperation', 'Weights', 'WeightsRole',
                      '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__',
                      '__package__', '__path__', '__spec__', '__version__', 'attr', 'bfloat16', 'bool',
                      'common_enter', 'common_exit', 'ctypes', 'float16', 'float32', 'fp8', 'get_builder_plugin_registry',
                      'get_nv_onnx_parser_version', 'get_plugin_registry', 'init_libnvinfer_plugins',
                      'int32', 'int4', 'int64', 'int8', 'nptype', 'os', 'sys',
                      'tensorrt', 'tensorrt_libs', 'uint8', 'value', 'volume', 'warnings']
import tensorflow as tf
import tensorrt
print(tf.config.list_physical_devices())
print(tensorrt.__file__)
print("tensorflow ver:", tf.__version__)
print("TensorRT ver:", tensorrt.__version__)
print("cuda ver:")
!nvcc --version
!nvidia-smi

['APILanguage', 'ActivationType', 'AllocatorFlag', 'BoundingBoxFormat', 'Builder', 'BuilderFlag', 'CalibrationAlgoType', 'DataType', 'DeviceType', 'DimensionOperation', 'Dims', 'Dims2', 'Dims3', 'Dims4', 'DimsExprs', 'DimsHW', 'DynamicPluginTensorDesc', 'ElementWiseOperation', 'EngineCapability', 'EngineInspector', 'ErrorCode', 'ErrorCodeTRT', 'ExecutionContextAllocationStrategy', 'FallbackString', 'FillOperation', 'GatherMode', 'HardwareCompatibilityLevel', 'IActivationLayer', 'IAlgorithm', 'IAlgorithmContext', 'IAlgorithmIOInfo', 'IAlgorithmSelector', 'IAlgorithmVariant', 'IAssertionLayer', 'IBuilderConfig', 'ICastLayer', 'IConcatenationLayer', 'IConditionLayer', 'IConstantLayer', 'IConvolutionLayer', 'ICudaEngine', 'IDebugListener', 'IDeconvolutionLayer', 'IDequantizeLayer', 'IDimensionExpr', 'IEinsumLayer', 'IElementWiseLayer', 'IErrorRecorder', 'IExecutionContext', 'IExprBuilder', 'IFillLayer', 'IGatherLayer', 'IGpuAllocator', 'IGpuAsyncAllocator', 'IGridSampleLayer', 'IHostMemory

In [20]:
import tensorflow as tf
!pip install nvidia-tensorrt
# Restart Runtime: After installation, restart your Colab runtime to apply changes.
# Enable TF-TRT optimization
tf.config.optimizer.set_experimental_options({"tf_trt_enable_native_ops": True})
!dpkg -l | grep nvidiatensorrt

In [39]:
# dir(tf)
!which python
!ls /usr/local/bin/t*
# !strace -e open,openat python -c "import tensorflow as tf" 2>&1 | grep "libnvinfer\|TF-TRT"
!dpkg -l | grep nvinfer
!pip install --upgrade --index-url https://pypi.ngc.nvidia.com nvidia-tensorrt
import tensorrt as trt
print(trt.__version__)
note="""
Looking in indexes: https://pypi.ngc.nvidia.com
Requirement already satisfied: nvidia-tensorrt in /usr/local/lib/python3.10/dist-packages (99.0.0)
Requirement already satisfied: tensorrt in /usr/local/lib/python3.10/dist-packages (from nvidia-tensorrt) (10.5.0)
Requirement already satisfied: tensorrt-cu12==10.5.0 in /usr/local/lib/python3.10/dist-packages (from tensorrt->nvidia-tensorrt) (10.5.0)
10.5.0
"""
import os
print(os.environ['LD_LIBRARY_PATH'])
# Add the directory to the LD_LIBRARY_PATH
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/lib/python3.10/dist-packages/tensorrt_libs'
print(os.environ['LD_LIBRARY_PATH'])
!echo $LD_LIBRARY_PATH
# !ls /usr/local/lib/python3.10/dist-packages

/usr/local/bin/python
/usr/local/bin/tabulate		      /usr/local/bin/tifffile
/usr/local/bin/tb-gcp-uploader	      /usr/local/bin/toco
/usr/local/bin/tensorboard	      /usr/local/bin/toco_from_protos
/usr/local/bin/test-importability.py  /usr/local/bin/torchrun
/usr/local/bin/tfds		      /usr/local/bin/tqdm
/usr/local/bin/tflite_convert	      /usr/local/bin/transformers-cli
/usr/local/bin/tf_upgrade_v2	      /usr/local/bin/ttx
/usr/local/bin/tiff2fsspec	      /usr/local/bin/typer
/usr/local/bin/tiffcomment
Looking in indexes: https://pypi.ngc.nvidia.com
10.5.0
/usr/lib64-nvidia:/usr/local/lib/python3.10/dist-packages
/usr/lib64-nvidia:/usr/local/lib/python3.10/dist-packages/tensorrt_libs
/usr/lib64-nvidia:/usr/local/lib/python3.10/dist-packages/tensorrt_libs


In [42]:
!ls /usr/local/lib/python3.10/dist-packages/tensorrt_libs
!cat /usr/local/lib/python3.10/dist-packages/tensorrt_libs/__init__.py

__init__.py				   libnvinfer_plugin.so.10  __pycache__
libnvinfer_builder_resource.so.10.5.0	   libnvinfer.so.10
libnvinfer_builder_resource_win.so.10.5.0  libnvonnxparser.so.10
#
# SPDX-FileCopyrightText: Copyright (c) 1993-2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

import ctypes
import glob
import os
import sys

CURDIR = os.path.realpath(os.path.dirname(__file__))


def try_load(library):
    try

In [40]:
%%time
# !pip install tensorrt
# Before training, it is recommended to first try the following code
# to verify everything is okay and memory is sufficient:
!python -m basic.cli --mode train --noload --debug
note_err="""
Requirement already satisfied: tensorrt in /usr/local/lib/python3.10/dist-packages (10.5.0)
Requirement already satisfied: tensorrt-cu12==10.5.0 in /usr/local/lib/python3.10/dist-packages (from tensorrt) (10.5.0)
2024-10-09 01:24:06.440695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 01:24:06.459772: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 01:24:06.469886: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 01:24:07.705035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig/basic/cli.py", line 5, in <module>
    from basic.main import main as m
  File "/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig/basic/main.py", line 14, in <module>
    from basic.model import get_multi_gpu_models
  File "/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/trivia_qa_bidaf_orig/basic/model.py", line 12, in <module>
    from my.tensorflow.rnn_cell import SwitchableDropoutWrapper, AttentionCell
ModuleNotFoundError: No module named 'my.tensorflow.rnn_cell'
CPU times: user 53.5 ms, sys: 11.4 ms, total: 64.8 ms
Wall time: 6.84 s

2024-08-18 23:19:24.895867: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
https://github.com/tensorflow/tensorflow/issues/64809
"""

2024-10-09 02:13:05.469103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 02:13:05.489416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 02:13:05.496007: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 02:13:06.666284: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals